In [189]:
import re
import json
from pymystem3 import Mystem
m = Mystem()

In [162]:
elines = []
with open('nefremova.txt', 'r', encoding='utf-8') as f:
    for line in f:
        elines.append(line)

In [163]:
pref = []
suff = []
for i, line in enumerate(elines):
    if '1. префикс' in line:
        if '.\n' not in elines[i-1]:
            pref.append(elines[i-1].replace('-\n', ''))
    elif '1. суффикс' in line:
        if '.п.' not in elines[i-1]:
            if '(' in elines[i-1]:
                suf = re.findall('.*?\(', elines[i-1])[0]
            else:
                suf = elines[i-1]
            suf = suf.replace('-', '').replace(' ', '').replace('(', '').replace('\n', '')
            if suf not in suff:
                suff.append(suf)
pref.extend(['ть', 'ться', 'ся'])

In [164]:
endings = ['ий', 'ый', 'ой', 'ая', 'ое', 'ие', 'а', 'я', 'ы', 'и', 'е', 'у', 'ю']

In [166]:
enouns = []
eadjs = []
everbs = []
eadvs = []
eoths = []
for i, line in enumerate(elines):
    if ' ж.' in line or ' м.' in line or ' ср.' in line:
        if '.\n' not in elines[i-1]:
            enouns.append(elines[i-1].replace('\n', ''))
    elif ' мн.' in line and i > 1:
        if '.\n' not in elines[i-1]:
            if m.lemmatize(elines[i-1])[0] == elines[i-1].replace('\n', ''):
                enouns.append(elines[i-1].replace('\n', ''))
    elif ' прил.' in line:
        if '.\n' not in elines[i-1]:
            eadjs.append(elines[i-1].replace('\n', ''))
    elif ' сов.' in line or ' несов.' in line:
        if '.\n' not in elines[i-1]:
            everbs.append(elines[i-1].replace('\n', ''))
    elif ' нареч.' in line:
        if '.\n' not in elines[i-1] and len(elines[i-1].split(' ')) < 2:
            eadvs.append(elines[i-1].replace('\n', ''))
    elif (
        ' союз' in line or 
        ' частица' in line or 
        ' местоим.' in line or 
        'числит.' in line or 
        'предлог' in line or
        'межд.' in line
        ) :
        if '.\n' not in elines[i-1] and len(elines[i-1].split(' ')) < 2:
            if len(elines[i-1].replace('\n', '')) > 2:
                eoths.append(elines[i-1].replace('\n', ''))

In [167]:
file_object = open('nests.json', 'r', encoding='utf-8')
json_nests = json.load(file_object)
file_object.close()

In [168]:
def root2rt(root):
    rt = root.replace('\n', '')
    rt = rt.replace('j', '')
    rt = rt.replace('I', '').replace('1', '').replace('2', '').replace('3', '')
    rt = rt.replace('<i>', '').replace('</i>', '')
    rt = rt.replace('(', '').replace(')', '')
    if ' ' in rt:
        rt = re.split(' ', rt)[0]
    elif '/' in rt:
        rt = re.split('/', rt)[-1]
    return rt

In [169]:
roots_dispose = []
roots_no_dispose = []
for nest in json_nests:
     for n in nest.values():
            for l in n:
                rt = root2rt(l['root'])
                if rt not in roots_dispose:
                    roots_dispose.append(rt)
                    roots_no_dispose.append(rt)

In [170]:
idshnik = 0
for n in json_nests:
    for k in n.keys():
        nest = n[k]
        for r in nest:
            if int(r['id']) > idshnik:
                idshnik = int(r['id'])
print(idshnik)

148997


In [171]:
def nprint(nest, root):
    trees ={}
    nr = 0
    for n in nest.values():
        nroot = []
        for l in n:
            if (
                root == l['root'] or
                l['root'].startswith(root + ' ') or
                l['root'].endswith('/' + root)
                ):
                trees[l['root']] = [l['root']]
                
        for r in trees:
            ts = [r]
            for l in n[::-1]:
                for t in ts:
                    if t in l['non-ambiguous'] or t in l['ambiguous']:
                        ts.append(l['root'])
            trees[r] = ts
        
        for r in trees:
            ts = []
            for l in n:
                if l['root'] in ts or l['root'] == r: 
                    for na in l['non-ambiguous']:
                        if na not in ts:
                            ts.append(na)
                    for a in l['ambiguous']:
                        if a not in ts:
                            ts.append(a)
            trees[r].extend(ts)
            
        for r in trees:
            for l in n:
                if l['root'] in trees[r]:
                    string = ''
                    for d in range(l['depth']):
                        string += '\t'
                    string += l['root']
                    print(string)

In [172]:
l = len(roots_dispose) + 1

while len(roots_dispose) < l:
    l = len(roots_dispose)
    print(l)
    
    for noun in enouns:
        if noun.replace('ё', 'е') in roots_no_dispose:
            if noun.replace('ё', 'е') in roots_dispose:
                roots_dispose.remove(noun.replace('ё', 'е'))
            enouns.remove(noun)

    for verb in everbs:
        if verb.replace('ё', 'е') in roots_no_dispose:
            if verb.replace('ё', 'е') in roots_dispose:
                roots_dispose.remove(verb.replace('ё', 'е'))
            everbs.remove(verb)

    for adj in eadjs:
        if adj.replace('ё', 'е') in roots_no_dispose:
            if adj.replace('ё', 'е') in roots_dispose:
                roots_dispose.remove(adj.replace('ё', 'е'))
            eadjs.remove(adj)

    for adv in eadvs:
        if adv.replace('ё', 'е') in roots_no_dispose:
            if adv.replace('ё', 'е') in roots_dispose:
                roots_dispose.remove(adv.replace('ё', 'е'))
            eadvs.remove(adv)

    for oth in eoths:
        if oth.replace('ё', 'е') in roots_no_dispose:
            if oth.replace('ё', 'е') in roots_dispose:
                roots_dispose.remove(oth.replace('ё', 'е'))
            eoths.remove(oth)


59569
33590
21633
16882
15661
15458
15434
15431


In [173]:
ealls = []
ealls.extend(enouns)
ealls.extend(eadjs)
ealls.extend(everbs)
ealls.extend(eadvs)
ealls.extend(eoths)
print(len(ealls))

73676


In [182]:
def add_nods(array, beens, idshnik):
    
    for jn in json_nests:
        for root in jn:
            rt2 = root2rt(root)
            if len(rt2) > 4:
                for e in endings:
                    if rt2.endswith(e) and len(rt2) - len(e) > 2:
                        rt2 = rt2[:(len(rt2) - len(e))]
            for n in array:
                if n in beens:
                    break

                n2 = n
                for e in endings:
                    if n2.endswith(e) and len(n2) - len(e) > 2:
                        n2 = n2[:(len(n2) - len(e))]
                
                if rt2 not in n2:
                    continue


        for nest in jn.values():
            
            rt2 = root2rt(nest[0]['root'])
            if len(rt2) > 4:
                for e in endings:
                    if rt2.endswith(e) and len(rt2) - len(e) > 2:
                        rt2 = rt2[:(len(rt2) - len(e))]
            for n in array:
                if n in beens or n not in ealls:
                    array.remove(n)
                    continue

                n2 = n
                for e in endings:
                    if n2.endswith(e) and len(n2) - len(e) > 2:
                        n2 = n2[:(len(n2) - len(e))]
                
                if rt2 not in n2:
                    continue
                
                for j, r in enumerate(nest[::-1]):
                    rt = root2rt(r['root'])
                    if len(rt) > 4:
                        for e in endings:
                            if rt.endswith(e) and len(rt) - len(e) > 2:
                                rt = rt[:(len(rt) - len(e))]


                    if n2.startswith(rt) and n2 != rt:
                        if n in ealls:
                            ealls.remove(n)
                        idshnik += 1

                        word1 = n
                        mettype1 = 'suf'
                        met1 = n[len(rt) - 1:]
                        ambiguous1 = []
                        non_ambiguous1 = []
                        depth1 = r['depth'] + 1
                        id1 = idshnik
                        
                        
                        if word1 in enouns:
                            PoS = 'S'
                        elif word1 in everbs:
                            PoS = 'V'
                        elif word1 in eadjs:
                            PoS = 'A'
                        elif word1 in eadvs:
                            PoS = 'ADV'
                        else:
                            try:
                                PoS = m.analyze(word1)[0]['analysis'][0]['gr'].split('=')[0].split(',')[0]
                            except (KeyError, IndexError, BrokenPipeError):
                                PoS = 'unknown'

                        root1 = {'root': word1, 'mettype': mettype1, 'met': met1,
                                  'ambiguous': ambiguous1, 'non-ambiguous': non_ambiguous1, 
                                  'depth': depth1,'id': id1, 'PoS': PoS}

                        for r2 in nest[::-1]:
                            if n in r2['ambiguous']:
                                r['ambiguous'].append(n)
                            elif n in r2['non-ambiguous']:
                                if n not in r['ambiguous']:
                                    r['ambiguous'].append(n)
                                r2['ambiguous'].append(n)
                                r2['non-ambiguous'].remove(n)
                        if n not in r['ambiguous']:
                            r['non-ambiguous'].append(n)

                        nest.insert(len(nest) - j, root1) 
                        beens.append(n)


                    elif rt.endswith(n2) and n2 != rt:
                        if n in ealls:
                            ealls.remove(n)
                        idshnik += 1

                        word1 = n
                        mettype1 = 'pref'
                        met1 = n2[:len(n2) - len(rt)]
                        ambiguous1 = []
                        non_ambiguous1 = []
                        depth1 = r['depth'] + 1
                        id1 = idshnik
                        idshnik += 1
                        
                        if word1 in enouns:
                            PoS = 'S'
                        elif word1 in everbs:
                            PoS = 'V'
                        elif word1 in eadjs:
                            PoS = 'A'
                        elif word1 in eadvs:
                            PoS = 'ADV'
                        else:
                            try:
                                PoS = m.analyze(word1)[0]['analysis'][0]['gr'].split('=')[0].split(',')[0]
                            except (KeyError, IndexError, BrokenPipeError):
                                PoS = 'unknown'

                        root1 = {'root': word1, 'mettype': mettype1, 'met': met1,
                                  'ambiguous': ambiguous1, 'non-ambiguous': non_ambiguous1, 
                                  'depth': depth1,'id': id1, 'PoS': PoS}

                        for r2 in nest[::-1]:
                            if n in r2['ambiguous']:
                                r['ambiguous'].append(n)
                            elif n in r2['non-ambiguous']:
                                if n not in r['ambiguous']:
                                    r['ambiguous'].append(n)
                                r2['ambiguous'].append(n)
                                r2['non-ambiguous'].remove(n)
                        if n not in r['ambiguous']:
                            r['non-ambiguous'].append(n)

                        nest.insert(len(nest) - j, root1) 
                        beens.append(n)

In [183]:
beens = []

In [ ]:
for each in ealls:
    add_nods2([each], beens, idshnik)
    print(len(ealls))

56249
56249
56248
56248
56247
56246
56245
56245
56245
56245
56245
56245
56244
56243
56243
56243
56242
56242
56242
56242
56242
56241
56241
56240
56239
56238
56238
56238
56238
56237
56236
56235
56235
56235
56234
56234
56234
56233
56233
56233
56233
56232
56232
56232
56232
56231
56231
56231
56231
56231
56231
56231
56231
56231
56230
56229
56229
56229
56229
56229
56229
56229
56229
56229
56229
56229
56229
56229
56229
56229
56229
56228
56228
56228
56228
56228
56228
56228
56228
56228
56228
56228
56227
56227
56226
56225
56225
56225
56225
56225
56225
56225
56225
56225
56225
56224
56224
56224
56223
56223
56223
56223
56223
56222
56221
56220
56220
56220
56220
56220
56220
56220
56220
56220
56220
56220
56220
56219
56218
56218
56218
56218
56218
56217
56217
56216
56216
56216
56216
56215
56215
56215
56215
56215
56215
56215
56215
56215
56215
56215
56215
56214
56214
56214
56213
56212
56211
56210
56209
56208
56207
56206
56205
56204
56203
56202
56201
56200
56199
56198
56197
56196
56195
56194
56193
56192
5619

55598
55598
55598
55598
55598
55598
55598
55598
55597
55596
55596
55596
55596
55596
55596
55596
55596
55595
55594
55593
55592
55592
55592
55591
55590
55589
55588
55587
55586
55585
55584
55583
55582
55581
55580
55579
55579
55579
55579
55578
55578
55578
55578
55578
55578
55578
55577
55577
55577
55576
55576
55576
55576
55576
55575
55574
55574
55574
55574
55574
55574
55574
55574
55574
55574
55574
55573
55572
55572
55572
55571
55571
55571
55571
55571
55570
55570
55570
55570
55569
55569
55569
55568
55567
55567
55567
55567
55566
55565
55564
55563
55562
55561
55560
55559
55558
55557
55556
55555
55555
55555
55555
55555
55555
55555
55555
55555
55555
55555
55555
55555
55555
55555
55555
55555
55555
55555
55554
55553
55552
55551
55550
55549
55548
55547
55547
55547
55546
55545
55544
55543
55542
55541
55540
55539
55538
55537
55536
55535
55534
55533
55532
55531
55530
55529
55528
55527
55526
55525
55524
55523
55522
55521
55520
55519
55518
55518
55518
55518
55518
55517
55517
55517
55517
55517
55516
5551

54971
54970
54969
54968
54967
54967
54966
54966
54966
54965
54964
54963
54962
54961
54960
54959
54959
54959
54959
54959
54958
54958
54958
54958
54958
54958
54958
54958
54958
54958
54957
54957
54957
54957
54957
54956
54955
54954
54953
54952
54951
54950
54949
54948
54947
54946
54945
54945
54945
54944
54943
54942
54942
54942
54942
54941
54941
54941
54940
54939
54939
54938
54938
54937
54936
54935
54935
54935
54935
54935
54935
54935
54935
54934
54933
54932
54931
54930
54929
54928
54927
54926
54925
54924
54923
54922
54921
54920
54919
54918
54917
54916
54916
54915
54915
54915
54915
54915
54915
54915
54915
54915
54914
54913
54912
54911
54910
54909
54908
54907
54906
54905
54904
54903
54902
54901
54900
54899
54898
54897
54896
54896
54896
54896
54896
54896
54895
54894
54893
54892
54891
54890
54889
54888
54888
54888
54888
54888
54888
54888
54888
54888
54888
54888
54888
54888
54888
54888
54888
54888
54888
54888
54888
54888
54888
54888
54888
54888
54887
54887
54887
54887
54886
54886
54886
54886
5488

54329
54328
54327
54326
54325
54324
54323
54322
54321
54320
54319
54318
54317
54316
54315
54314
54313
54312
54311
54310
54309
54308
54307
54306
54305
54304
54303
54302
54301
54300
54299
54298
54297
54296
54295
54294
54293
54292
54291
54290
54289
54288
54287
54286
54285
54284
54283
54282
54281
54280
54279
54278
54277
54276
54275
54274
54273
54272
54271
54270
54269
54268
54267
54266
54265
54264
54263
54262
54261
54260
54259
54258
54257
54256
54255
54254
54253
54252
54251
54250
54249
54249
54248
54247
54246
54245
54244
54243
54242
54241
54240
54239
54238
54237
54236
54235
54234
54233
54232
54231
54230
54229
54228
54227
54226
54225
54224
54223
54222
54221
54220
54220
54219
54218
54217
54216
54215
54214
54213
54212
54211
54210
54209
54208
54207
54206
54205
54204
54203
54202
54201
54200
54199
54198
54197
54196
54195
54194
54193
54192
54191
54190
54189
54188
54187
54186
54185
54184
54183
54182
54181
54180
54179
54178
54177
54176
54175
54174
54173
54172
54171
54170
54169
54168
54167
54166
5416

53619
53618
53617
53617
53617
53617
53617
53617
53617
53617
53616
53615
53614
53613
53612
53611
53611
53610
53609
53608
53607
53606
53605
53605
53605
53605
53605
53605
53605
53604
53603
53602
53601
53600
53600
53600
53599
53598
53597
53596
53595
53595
53595
53595
53595
53594
53594
53594
53594
53594
53594
53594
53593
53593
53593
53593
53593
53593
53592
53592
53592
53592
53592
53592
53591
53591
53591
53591
53591
53590
53589
53589
53589
53588
53587
53586
53586
53586
53586
53586
53586
53586
53586
53586
53586
53586
53586
53586
53586
53586
53586
53586
53586
53586
53586
53586
53586
53586
53586
53586
53586
53586
53585
53584
53583
53582
53581
53580
53579
53578
53577
53577
53577
53577
53577
53576
53575
53574
53573
53573
53573
53572
53571
53570
53569
53568
53567
53566
53565
53564
53563
53562
53561
53560
53559
53559
53559
53559
53559
53558
53557
53556
53555
53554
53553
53552
53551
53550
53549
53549
53548
53547
53546
53545
53544
53543
53542
53541
53540
53539
53538
53537
53536
53535
53534
53533
5353

52764
52763
52762
52761
52760
52759
52758
52757
52756
52755
52754
52753
52752
52751
52750
52749
52748
52747
52746
52745
52744
52743
52742
52741
52740
52739
52738
52737
52736
52735
52734
52733
52732
52731
52730
52729
52728
52727
52726
52725
52724
52723
52722
52721
52720
52719
52718
52717
52716
52715
52714
52713
52712
52711
52710
52709
52708
52707
52706
52705
52704
52703
52702
52701
52700
52699
52698
52697
52696
52695
52694
52693
52692
52691
52691
52690
52689
52688
52687
52686
52685
52684
52683
52682
52681
52680
52679
52678
52677
52676
52675
52674
52673
52672
52671
52670
52669
52668
52667
52666
52665
52664
52663
52662
52661
52660
52659
52658
52657
52656
52655
52654
52653
52652
52651
52650
52649
52648
52647
52646
52645
52644
52643
52642
52641
52640
52639
52638
52637
52636
52635
52634
52633
52632
52631
52630
52629
52628
52627
52626
52625
52624
52623
52622
52621
52620
52619
52618
52617
52616
52615
52614
52613
52612
52611
52611
52611
52610
52609
52608
52607
52606
52605
52604
52603
52602
5260

In [184]:
new_cells = []
for i, word in enumerate(ealls):
    w = word
    original = True
    for e in endings:
        if w.endswith(e):
            w = w[:(len(w) - len(e))]
    for p in pref:
        if w.startswith(p):
            original = False
    for s in suff:
        if w.endswith(s):
            original = False
    if original == True:
        new_cells.append(word)
        
    elif len(ealls) - 1 > i > 0:
        if ealls[i + 1].startswith(word) and not ealls[i - 1].startswith(word):
            new_cells.append(word)

In [185]:
print(len(new_cells))

13


In [186]:
for c in new_cells:
    if c in ealls:
        ealls.remove(c)
    if ' ' not in c:
        try:
            PoS = m.analyze(c)[0]['analysis'][0]['gr'].split('=')[0].split(',')[0]
        except (KeyError, IndexError, BrokenPipeError):
            if c in enouns:
                PoS = 'S'
            elif c in everbs:
                PoS = 'V'
            elif c in eadjs:
                PoS = 'A'
            elif c in eadvs:
                PoS = 'ADV'
            else:
                PoS = 'unknown'


        json_nests.append({c:[{'root': c, 'mettype': 'else', 'met': 'none',
                              'ambiguous': [], 'non-ambiguous': [], 
                              'depth': 0,'id': idshnik + 1, 
                               'PoS': PoS}]})
        idshnik += 1
    

In [190]:
for j in json_nests[-1]:
    elast = j
for eword in ealls:
    if eword in new_cells:
        ealls.remove(eword)
        if ' ' not in eword:
            try:
                PoS = m.analyze(eword)[0]['analysis'][0]['gr'].split('=')[0].split(',')[0]
            except (KeyError, IndexError, BrokenPipeError):
                if eword in enouns:
                    PoS = 'S'
                elif eword in everbs:
                    PoS = 'V'
                elif eword in eadjs:
                    PoS = 'A'
                elif eword in eadvs:
                    PoS = 'ADV'
                else:
                    PoS = 'unknown'
                    
            json_nests.append({eword:[{'root': eword, 'mettype': 'else', 'met': 'none',
                              'ambiguous': [], 'non-ambiguous': [], 
                              'depth': 0,'id': idshnik + 1, 
                                'PoS': PoS}]})
            idshnik += 1
        elast = eword
        
    else:
        for r in json_nests[-1][elast]:
            n = eword
            n2 = eword
            for e in endings:
                if n2.endswith(e) and len(n2) - len(e) > 2:
                    n2 = n2[:(len(n2) - len(e))]


            rt = root2rt(r['root'])
            if len(rt) > 4:
                for e in endings:
                    if rt.endswith(e) and len(rt) - len(e) > 2:
                        rt = rt[:(len(rt) - len(e))]


            if n2.startswith(rt) and n2 != rt:
                if n in ealls:
                    ealls.remove(n)
                idshnik += 1

                word1 = n
                mettype1 = 'suf'
                met1 = n[len(rt) - 1:]
                ambiguous1 = []
                non_ambiguous1 = []
                depth1 = r['depth'] + 1
                id1 = idshnik
                
                try:
                    PoS = m.analyze(word1)[0]['analysis'][0]['gr'].split('=')[0].split(',')[0]
                except (KeyError, IndexError, BrokenPipeError):
                    if word1 in enouns:
                        PoS = 'S'
                    elif word1 in everbs:
                        PoS = 'V'
                    elif word1 in eadjs:
                        PoS = 'A'
                    elif word1 in eadvs:
                        PoS = 'ADV'
                    else:
                        PoS = 'unknown'
                    

                root1 = {'root': word1, 'mettype': mettype1, 'met': met1,
                        'ambiguous': ambiguous1, 'non-ambiguous': non_ambiguous1, 
                        'depth': depth1,'id': id1, 'PoS': m.analyze(root2rt(word1))[0]['analysis'][0]['gr'].split('=')[0].split(',')[0]}

                for r2 in json_nests[-1][elast][::-1]:
                    if n in r2['ambiguous']:
                        r['ambiguous'].append(n)
                    elif n in r2['non-ambiguous']:
                        if n not in r['ambiguous']:
                            r['ambiguous'].append(n)
                        r2['ambiguous'].append(n)
                        r2['non-ambiguous'].remove(n)
                if n not in r['ambiguous']:
                    r['non-ambiguous'].append(n)

                json_nests[-1][elast].append(root1) 


            elif rt.endswith(n2) and n2 != rt:
                if n in ealls:
                    ealls.remove(n)
                idshnik += 1

                word1 = n
                mettype1 = 'pref'
                met1 = n2[:len(n2) - len(rt)]
                ambiguous1 = []
                non_ambiguous1 = []
                depth1 = r['depth'] + 1
                id1 = idshnik
                idshnik += 1
                
                try:
                    PoS = m.analyze(word1)[0]['analysis'][0]['gr'].split('=')[0].split(',')[0]
                except (KeyError, IndexError, BrokenPipeError):
                    if word1 in enouns:
                        PoS = 'S'
                    elif word1 in everbs:
                        PoS = 'V'
                    elif word1 in eadjs:
                        PoS = 'A'
                    elif word1 in eadvs:
                        PoS = 'ADV'
                    else:
                        PoS = 'unknown'
                    

                root1 = {'root': word1, 'mettype': mettype1, 'met': met1,
                            'ambiguous': ambiguous1, 'non-ambiguous': non_ambiguous1, 
                            'depth': depth1,'id': id1, 'PoS': PoS}

                for r2 in json_nests[-1][elast][::-1]:
                    if n in r2['ambiguous']:
                        r['ambiguous'].append(n)
                    elif n in r2['non-ambiguous']:
                        if n not in r['ambiguous']:
                            r['ambiguous'].append(n)
                        r2['ambiguous'].append(n)
                        r2['non-ambiguous'].remove(n)
                if n not in r['ambiguous']:
                    r['non-ambiguous'].append(n)

                json_nests[-1][elast].append(root1) 


In [191]:
print(len(ealls))

56249


In [71]:
file_object = open('nests_new.json', 'w', encoding='utf-8')
json.dump(json_nests, file_object, ensure_ascii=False)
file_object.close()

In [192]:
def add_nods2(array, beens, idshnik):
    for n in array:


        n2 = n
        for e in endings:
            if n2.endswith(e) and len(n2) - len(e) > 2:
                n2 = n2[:(len(n2) - len(e))]


        for jn in json_nests:
            if n in beens:
                break


            for root in jn:
                rt2 = root2rt(root)
                if len(rt2) > 4:
                    for e in endings:
                        if rt2.endswith(e) and len(rt2) - len(e) > 2:
                            rt2 = rt2[:(len(rt2) - len(e))] 
            if rt2 not in n2:
                continue


            for nest in jn.values():
                for j, r in enumerate(nest[::-1]):
                    rt = root2rt(r['root'])
                    if len(rt) > 4:
                        for e in endings:
                            if rt.endswith(e) and len(rt) - len(e) > 2:
                                rt = rt[:(len(rt) - len(e))]


                    if n2.startswith(rt) and n2 != rt:
                        if n in ealls:
                            ealls.remove(n)
                        idshnik += 1

                        word1 = n
                        mettype1 = 'suf'
                        met1 = n[len(rt) - 1:]
                        ambiguous1 = []
                        non_ambiguous1 = []
                        depth1 = r['depth'] + 1
                        id1 = idshnik
                        
                        
                        if word1 in enouns:
                            PoS = 'S'
                        elif word1 in everbs:
                            PoS = 'V'
                        elif word1 in eadjs:
                            PoS = 'A'
                        elif word1 in eadvs:
                            PoS = 'ADV'
                        else:
                            try:
                                PoS = m.analyze(word1)[0]['analysis'][0]['gr'].split('=')[0].split(',')[0]
                            except (KeyError, IndexError, BrokenPipeError):
                                PoS = 'unknown'

                        root1 = {'root': word1, 'mettype': mettype1, 'met': met1,
                                  'ambiguous': ambiguous1, 'non-ambiguous': non_ambiguous1, 
                                  'depth': depth1,'id': id1, 'PoS': PoS}

                        for r2 in nest[::-1]:
                            if n in r2['ambiguous']:
                                r['ambiguous'].append(n)
                            elif n in r2['non-ambiguous']:
                                if n not in r['ambiguous']:
                                    r['ambiguous'].append(n)
                                r2['ambiguous'].append(n)
                                r2['non-ambiguous'].remove(n)
                        if n not in r['ambiguous']:
                            r['non-ambiguous'].append(n)

                        nest.insert(len(nest) - j, root1) 
                        beens.append(n)


                    elif rt.endswith(n2) and n2 != rt:
                        if n in ealls:
                            ealls.remove(n)
                        idshnik += 1

                        word1 = n
                        mettype1 = 'pref'
                        met1 = n2[:len(n2) - len(rt)]
                        ambiguous1 = []
                        non_ambiguous1 = []
                        depth1 = r['depth'] + 1
                        id1 = idshnik
                        idshnik += 1
                        
                        if word1 in enouns:
                            PoS = 'S'
                        elif word1 in everbs:
                            PoS = 'V'
                        elif word1 in eadjs:
                            PoS = 'A'
                        elif word1 in eadvs:
                            PoS = 'ADV'
                        else:
                            try:
                                PoS = m.analyze(word1)[0]['analysis'][0]['gr'].split('=')[0].split(',')[0]
                            except (KeyError, IndexError, BrokenPipeError):
                                PoS = 'unknown'

                        root1 = {'root': word1, 'mettype': mettype1, 'met': met1,
                                  'ambiguous': ambiguous1, 'non-ambiguous': non_ambiguous1, 
                                  'depth': depth1,'id': id1, 'PoS': PoS}

                        for r2 in nest[::-1]:
                            if n in r2['ambiguous']:
                                r['ambiguous'].append(n)
                            elif n in r2['non-ambiguous']:
                                if n not in r['ambiguous']:
                                    r['ambiguous'].append(n)
                                r2['ambiguous'].append(n)
                                r2['non-ambiguous'].remove(n)
                        if n not in r['ambiguous']:
                            r['non-ambiguous'].append(n)

                        nest.insert(len(nest) - j, root1) 
                        beens.append(n)


            if n not in ealls:
                break

